<a href="https://colab.research.google.com/github/mariaaseret/MDE/blob/main/Aula8_MariaTeresa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação das Bibliotecas

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

# Carregamento dos Dados:

## Estrutura dos dados:

* UNIQUEID: Identificador único para cada observação.
* SCHOOL: A escola onde a observação foi feita.
* Class: A classe onde a observação foi feita.
* GRADE: O grau escolar dos estudantes.
* CODER: Identificador do avaliador que fez a observação.
* STUDENTID: Identificador único do estudante.
* Gender: Gênero do estudante.
* OBSNUM: Número da observação para o estudante.
* totalobs-forsession: Contagem total de observações para a sessão.
* Activity: A atividade que estava sendo realizada.
* ONTASK: Variável alvo, indicando se o estudante estava engajado (Y=Yes) ou distraído (N=No).
* TRANSITIONS: Número de transições de atividades.
* NumACTIVITIES: Número de atividades diferentes observadas.
* FORMATchanges: Mudanças de formato da atividade.
* NumFORMATS: Número de formatos diferentes de atividades.
* Obsv/act: Observação por atividade.
* Transitions/Durations: Razão entre transições e durações.
* Total Time: Tempo total gasto na atividade.

------

## Segundo o artigo as variáveis utilizadas são:

As variáveis utilizadas como preditoras foram separadas em duas categorias: característica do aluno (gender, grade -- gênero e série) e design instrucional (Activity, Transitions/Durations -- formato da instrução e Transições/Duração).



In [10]:
# Carregar dados
data = pd.read_csv('/content/a1-in.csv')

Pré-processamento e Seleção de Variáveis

In [5]:
# Selecionando as variáveis relevantes conforme a tarefa
features = data[['Gender', 'GRADE', 'Activity', 'Transitions/Durations']]
target = data['ONTASK']

# Conversão de 'ONTASK' para binário
target = target.map({'Y': 1, 'N': 0})

Divisão dos Dados em Treino e Teste

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

Construção do Pipeline

In [7]:
# Pipeline para tratamento numérico e categórico
numeric_features = ['GRADE', 'Transitions/Durations']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Gender', 'Activity']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combinar transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Modelos a serem avaliados
classifiers = [
    ('logreg', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC())
]

# Grids de hiperparâmetros para GridSearchCV
param_grids = {
    'logreg': {'classifier__C': [0.1, 1, 10]},
    'rf': {'classifier__n_estimators': [50, 100, 200]},
    'dt': {'classifier__max_depth': [None, 10, 20, 30]},
    'svc': {'classifier__C': [0.1, 1, 10]}
}

Treinamento e Otimização com GridSearchCV

In [8]:
best_models = {}
for name, classifier in classifiers:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', classifier)])
    # GridSearchCV para encontrar os melhores parâmetros
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Melhor modelo para {name}: {grid_search.best_params_}")


Melhor modelo para logreg: {'classifier__C': 0.1}
Melhor modelo para rf: {'classifier__n_estimators': 200}
Melhor modelo para dt: {'classifier__max_depth': 10}
Melhor modelo para svc: {'classifier__C': 10}


Avaliação dos Modelos

In [9]:
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"Relatório de classificação para o modelo {name}:")
    print(classification_report(y_test, y_pred))

Relatório de classificação para o modelo logreg:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1790
           1       0.68      1.00      0.81      3757

    accuracy                           0.68      5547
   macro avg       0.34      0.50      0.40      5547
weighted avg       0.46      0.68      0.55      5547



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Relatório de classificação para o modelo rf:
              precision    recall  f1-score   support

           0       0.57      0.18      0.27      1790
           1       0.71      0.93      0.80      3757

    accuracy                           0.69      5547
   macro avg       0.64      0.56      0.54      5547
weighted avg       0.66      0.69      0.63      5547

Relatório de classificação para o modelo dt:
              precision    recall  f1-score   support

           0       0.57      0.16      0.25      1790
           1       0.70      0.94      0.80      3757

    accuracy                           0.69      5547
   macro avg       0.63      0.55      0.52      5547
weighted avg       0.66      0.69      0.62      5547

Relatório de classificação para o modelo svc:
              precision    recall  f1-score   support

           0       0.72      0.01      0.03      1790
           1       0.68      1.00      0.81      3757

    accuracy                           0.68   

## Análise de Melhor Modelo

Após a avaliação de diversos modelos de classificação binária, o modelo de **Floresta Aleatória (Random Forest)** se destacou como o mais promissor para a tarefa de prever comportamentos on-task e off-task em ambientes educacionais. Os resultados obtidos foram comparados baseando-se em métricas de precisão, recall e F1-score para ambas as classes (on-task e off-task).

### Resultados do Modelo de Floresta Aleatória:
- **Precisão para on-task (1):** 71%
- **Recall para on-task (1):** 93%
- **F1-score para on-task (1):** 80%
- **Precisão para off-task (0):** 57%
- **Recall para off-task (0):** 18%
- **F1-score para off-task (0):** 27%

### Justificativa para a Escolha:
O modelo de Floresta Aleatória demonstrou um equilíbrio relativamente melhor entre precisão e recall para ambas as classes em comparação com os outros modelos testados. Apesar de os valores de recall e F1-score para a classe off-task ainda serem baixos, este modelo conseguiu atingir uma precisão e F1-score consideráveis para a classe on-task, que compõe a maioria dos casos.

### Importância das Métricas:
O **recall** é crucial, especialmente para a classe off-task, pois indica a capacidade do modelo de capturar todos os casos relevantes de desengajamento. Um alto recall na classe off-task significa que o sistema é capaz de identificar eficientemente os alunos que precisam de atenção adicional, possibilitando intervenções educacionais mais eficazes.

O **F1-score** também é uma métrica importante, pois equilibra a precisão e o recall, sendo particularmente útil em situações com classes desbalanceadas como esta. O F1-score mais alto para on-task mostra que o modelo é competente em identificar o engajamento dos alunos, mas é necessário melhorar o F1-score para off-task para tornar o modelo mais útil em ambientes educacionais práticos.
